### Step 1: Import Libraries

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import json

### Step 2: Set the News Timeframe to the Last 30 Days

In [7]:
now = datetime.today()
one_month_ago = now - timedelta(days=30) + timedelta(minutes=1)
time_from = one_month_ago.strftime("%Y%m%dT%H%M")
time_to = now.strftime("%Y%m%dT%H%M")


In [8]:
API_KEY = "CWMJJ57KWGHTBMAV"

### Step 3: Load the S&P 500 Tickers from a CSV File

In [14]:
df_tickers = pd.read_csv("../data/S&P500.csv", encoding='latin1')
tickers = df_tickers['Symbol'].dropna().unique()[6:11]  # First 5 to test

tickers

array(['AMD', 'AES', 'AFL', 'A', 'APD'], dtype=object)

### Step 4: List to Store All News

In [15]:
all_articles = []

### Step 5: Fetch and Preview Articles for the First 20 Tickers

In [16]:
for i, ticker in enumerate(tickers):
    url = (
        f"https://www.alphavantage.co/query?"
        f"function=NEWS_SENTIMENT"
        f"&tickers={ticker}"
        f"&time_from={time_from}"
        f"&time_to={time_to}"
        f"&limit=50"
        f"&apikey={API_KEY}"
    )
    
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        feed = resp.json().get("feed", [])

        for article in feed:
            all_articles.append({
                "Ticker": ticker,
                "Title": article.get("title"),
                "Source": article.get("source"),
                "PublishedAt": article.get("time_published"),
                "Sentiment": article.get("overall_sentiment_label"),
                "Sentiment Score": article.get("overall_sentiment_score"),
                "URL": article.get("url")
            })

        print(f"[{i+1}/5] ✅ {ticker} — {len(feed)} articles")
        time.sleep(12)  # respects 5 req/min limit

    except Exception as e:
        print(f"[{i+1}/5] ❌ {ticker} — Error: {e}")

[1/5] ✅ AMD — 50 articles
[2/5] ✅ AES — 3 articles
[3/5] ✅ AFL — 11 articles
[4/5] ✅ A — 10 articles
[5/5] ✅ APD — 2 articles


### Step 6: Store as CSV File

In [17]:
df = pd.DataFrame(all_articles)
df.to_csv("test_5_ticker_news.csv", index=False)
print("✅ Saved: test_5_ticker_news.csv")

✅ Saved: test_5_ticker_news.csv
